# Mapping Coastal Building Footprints

This notebook was written for the CoCliCo meet-up in Orléans,  October 2022, to demonstrate the CoCliCo STAC API Demonstration. 

The purpose of this notebook is to provide an example of how data that is available in the CoCliCo STAC catalog can be combined with data from other parties to do cloud-based data analysis.

In this dataset we combine the coastal mask (Daniel Lincke, WCF, 2022) with buildings footprints [dataset](https://github.com/microsoft/GlobalMLBuildingFootprints) from Microsoft for The Netherlands.

In [ ]:
import os
import pathlib
import sys

# make coclico library importable by appending src from project root to path
cwd = pathlib.Path().resolve()
PROJ_ROOT = os.path.dirname(cwd)

sys.path.append(os.path.join(PROJ_ROOT, "src"))

## Python tools

Below we import some python packages that are often used when working with geospatial data that are hosted in the cloud. These libraries are listed a `environment.yml` file that is available in the root of coclico workbench repository. When the conda package manager is available on the machine (or better: mamba), a new environment with these packages can be created by running `conda create -n coclico -f environment.yml`

In [ ]:
!cat "{PROJ_ROOT}/environment.yml"

In [ ]:
import dask
import dask_geopandas
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pystac_client
import rasterio
import rioxarray
import shapely
import xarray as xr

## Connect to the coclico-stac catalog

The STAC catalog is currenlty hosted in our data bucket at https://storage.googleapis.com/dgds-data-public/coclico/coclico-stac/catalog.json. In future we could (and maybe should) move this to  https://coclicoservices.eu/api/stac/v1

With `pystac` the json files that describe the coclico data following STAC specifications can be read into a Python object that is browsable and searchable. 

In [ ]:
catalog = pystac_client.Client.open(
    "https://storage.googleapis.com/dgds-data-public/coclico/coclico-stac/catalog.json"
)
catalog

### Structure of STAC catalogues 

- Catalog: A STAC catalog is a top-level object that logically groups other Catalog, Collection, and item objects. 
- Collection: A STAC Collection provides additional information about a spatio-temporal collection of data. 
- Item: a STAC Item represents an atomic collection of inseparable data and metadata.
- Asset: Links to data. 

For more please refer to the [documentation](https://stacspec.org/en/about/stac-spec/)

## Browse the catalog

There are several methods to browse the STAC catalog. Noteworthy is the search method `catalog.search()`, which is able to search for bounding boxes or tags. When datasets are created by the provides they can provide a list of "tags" which can be added to the stac. In that way users can for example browse all datasets related to "sea-level-rise". 

In [ ]:
# Here we just list all collections that are present in the catalog
list(catalog.get_children())

### Get the items of a given dataset
Collections may include several items, for example, each representing one Cloud Optimized GeoTiff. Here we make a list of STAC items that are included in the Coastal Mask collection. 

In [ ]:
# we can now browse the datasets by interacting with the STAC catalog
items = list(catalog.get_child("cm").get_all_items())

### What are STAC items?

Nothing more than text files describing what's in the data!

In [ ]:
import sys

print(f"{sys.getsizeof(items[0].to_dict())} bytes")
items[0].to_dict()

In [ ]:
items[0].to_dict()["assets"]["cm"]["href"]

## Particular advantage of working with STAC: indexing datasets without having to read the data


The aim here is to retrieve a list of items that contain data of our region of interest. So first we have to define a region of interest 

In the cell below, we import a function from the coclico-workbench tools for creating a bounding box geometry, wrapped inside a GeoPandas GeoDataFrame. This geometry is created from a list of bbox coordinates (yuor ROI). 

Working with GeoPandas dataframes is often handy because the library provides spatial indices to the dataframe, thus, data can be accessed using rtree functionalities.  

In [ ]:
from coclico.utils.geo import geo_bbox

roi = geo_bbox(
    [5.2774, 53.0838, 7.1232, 53.5638]
)  # rois can be drawn using for example https://boundingbox.klokantech.com/
roi.explore()

### Get the bounding box for each of the STAC items 

All the STAC items contain a property that describes the bounding bbox of the data. All these properties are converted to GeoPandas dataframes, which are concatenated into one dataframe.

This dataframe describes the spatial extent per bounding box. When the source data (`coastal_mask.tif`) was converted to Cloud Optimized GeoTiff's  we chose a size of 512 x 512 pixels. The coastal mask was derived from the SRTM Dem, so each pixel equivalents approximately 90 meters. Thus, one of these bounding boxes span an area of 45 x 45 km, although the outer boxes may contain less pixels.  

Note, here we construct the spatial extent from the STAC items by reading the bbox property in each item. In future, when search functions are implemenented, this will be even easier, as we can use the `pystac_client.Client().search()` method that can handle geometries such as bounding boxes. 

In [ ]:
bboxes = pd.concat([geo_bbox(i.to_dict()["bbox"]) for i in items])
bboxes = bboxes.reset_index(drop=True)
bboxes.explore()

### Get items for region of interest

Suppose that only that from a certain region is required. In such scenario a bounding box of the region of interest (ROI) can be created using several gis tools, [also online](https://boundingbox.klokantech.com). With GeoPandas it is now easy to get the bboxes that match the ROI. 

In [ ]:
bboxes_roi = gpd.sjoin(bboxes, roi)[bboxes.columns]
bboxes_roi.explore()

In [ ]:
# obtain STAC items that cover the ROI
items_roi = [items[i] for i in bboxes_roi.index]

In [ ]:
# inspect the first item
items_roi[0]

## Lazy loading with Dask 

Dask is useful to computations with data that exceed available memory on a machine because it is using lazy execution and iterating over the workload per partition. 

Insetad of listing all items we now list all hrefs available in the Coastal mask. These hrefs point to the blob objects in the data bucket (in this case COG's). Many libraries are able to read data directly from these hrefs (e.g., `Pandas, GeoPandas, Xarray, Dask and Dask_GeoPandas`. The list of href's is read with dase and opened using the rasterio backend. 

In [ ]:
cm_items = list(catalog.get_child("cm").get_all_items())
cm_hrefs = [i.assets["cm"].href for i in cm_items]
cm_hrefs[0]

## Launch a Dask cluster

Bit beyond the scope of this talk..

In [ ]:
client = dask.distributed.Client(threads_per_worker=1, local_directory="/tmp")
print(client)
print(client.dashboard_link)

In [ ]:
%%time
@dask.delayed
def lazy_open(href):
    chunks = dict(band=1, x=512, y=512)
    return xr.open_dataset(href, chunks=chunks, engine="rasterio")


das = dask.compute(*[lazy_open(href) for href in cm_hrefs])

In [ ]:
%%time
dataset = xr.combine_by_coords(das).compute()

In [ ]:
%%time
import hvplot.xarray  # qa

dataset.squeeze("band").hvplot.image(rasterize=True, x="x", y="y", aspect="equal")

## Combine the coastal mask with building footprints

Microsoft planetary computer hosts a building footprint dataset obtained from very-high resolution satellite imagery using NN's. Here we combine these building footprints with the coastal mask to calculate the building area per flood risk unit. 

Interacting with Planetary computer is beyond the scope of this tutorial, so a sample of the buildings is stored in the coclico data bucket for demonstration purposes.

In [ ]:
import dask_geopandas

buildings_ddf = dask_geopandas.read_parquet(
    "gs://dgds-data-public/coclico/building_footprints_netherlands/"
)

buildings_ddf["building_area"] = buildings_ddf.to_crs(3857).geometry.area / 1e6
buildings_ddf

## Building footprints from MS Planetary computer

For some regions these data are very accurate. Especially in remote areas they can be useful. On the other hand.. other regions might be missing without clear cause. It woud be useful to integrate this dataset with OSM building footprints

In [ ]:
buildings_ddf.partitions[0].compute().iloc[:3000].explore()

### Vectorize coastal mask data

The coastal mask data are rasters, but to overlay this with the building footprints dataset, either the rasters data has to be vectorized or the building data has to be rasterized. Here we will vectorize the coastal mask dataset. 

In [ ]:
%%time

from geopandas.array import GeometryDtype


@dask.delayed
def lazy_vectorize(href):
    with rasterio.open(href) as src:
        # TODO: change default nodata values in source tif because mask=True does not work now
        # currently nodata values are float("nan")
        data = src.read(1)  # could be src.read(1, mask=True)

        # returns boolean array with true's for nan values
        mask = np.isnan(data)

        # Create vectors around the raster values that are not float("nan") (masked).
        # The rasterio.features.shapes function from rasterio returns dictionaries in geojson
        # format of the geometry with the associated value (will always be 1 in this
        # dataset. Shapely.geometry.shape can be used to convert the geojson dictionary
        # to a geometry. The transform (affine) is used to map the raster values to the
        # cartesian coordinates.
        shape_generator = [
            (shapely.geometry.shape(s), v)
            for s, v in rasterio.features.shapes(
                data, mask=~mask, transform=src.transform
            )
        ]

        # two different methods (doesn't really make a difference):
        # 1) generator to dictionary..
        # store the geometries and associated values in a dictionary
        data = dict(zip(["geometry", "value"], zip(*shape_generator)))

        # cannot construct geodataframe from empty dictionary, so return empty (works with dask)
        if not data:
            return

        gdf = gpd.GeoDataFrame.from_dict(data, crs=src.crs)

    #         # 2) or unpack generator in two lists and construct dataframe from that at
    #         vectors = list(shape_generator)
    #         # Extract the polygon coordinates and values from the list
    #         polygons = [polygon for polygon, _ in vectors]
    #         values = [value for _, value in vectors]

    #         # Create a geopandas dataframe populated with the polygon shapes
    #         gdf = gpd.GeoDataFrame(data={"values": values}, geometry=polygons, crs=src.crs)

    return gdf


meta = {"geometry": GeometryDtype, "value": float}
coastal_mask_vec = dask.compute(*[lazy_vectorize(href) for href in cm_hrefs], meta=meta)
coastal_mask = pd.concat(coastal_mask_vec)
coastal_mask.explore()

## Preprocess coastal mask

Drop small regions and add a unique name.

In [ ]:
import uuid

coastal_mask = pd.concat(coastal_mask_vec)
coastal_mask = coastal_mask.to_crs(3857)  # areas should be computed in p
coastal_mask["cm_area"] = coastal_mask.area / 1e6  # compute area in sq km
coastal_mask = coastal_mask.loc[
    coastal_mask["cm_area"] > 1
].copy()  ## only keep flood risk units greater than 1 sq km
coastal_mask = coastal_mask.reset_index(drop=True)
coastal_mask["cm_name"] = [
    "".join(str(uuid.uuid4()).split("-")) for i in range(len(coastal_mask.index))
]
coastal_mask = coastal_mask.to_crs(4326)
coastal_mask.head()

### Spatial join using Dask Geopandas

Load the vectorized coastal maks into Dask GeoPandas so that its spatial join methods can be used.  

In [ ]:
coastal_mask_ddf = dask_geopandas.from_geopandas(coastal_mask, npartitions=1)

### Compute building footprints per coastal mask region

The computation is only triggered when we call `compute()`. The dashboard link provides a GUI monitor the task, including things such as task bars

In [ ]:
%%time
print(client.dashboard_link)
joined_ddf = buildings_ddf.sjoin(coastal_mask_ddf)
joined = joined_ddf.compute()
joined = joined.loc[joined.geometry.is_valid].drop("index_right", axis="columns")

### Visualize results for ROI

The data will be plotted using a datashader as the ROI still contains 350K buildings. The geometries have to be converted to a Spatial pandas GeoDataFrame before datashader is able to handle them. 

In [ ]:
joined_roi = gpd.sjoin(joined, roi)[joined.columns]

## Make a visualization of the results

Here we only show the overlay for the region of interest, as GeoViews requires SpatialPandas GeoDatFrames. Converting the geometries to from GeoPandas GeoDataFrame's to SpatialPandas geometries takes some time without dask. If you want the image for the whole Netherlands, change to `sp_joined_roi = sp.GeoDataFrame(joined)`

In [ ]:
import spatialpandas as sp

sp_coastal_mask_roi = sp.GeoDataFrame(
    gpd.sjoin(coastal_mask, roi)[coastal_mask.columns]
)
sp_joined_roi = sp.GeoDataFrame(joined)

In [ ]:
import colorcet as cc
import geoviews as gv
import holoviews.operation.datashader as hd

gv.extension("bokeh")
base_layer_hv = gv.tile_sources.EsriTerrain.opts(
    width=1200, height=750, bgcolor="black"
)
coastal_mask_hv = gv.Polygons(sp_coastal_mask_roi.geometry).opts(
    fill_color="gray", alpha=0.3, line_color=None
)
building_footprints_hv = hd.rasterize(gv.Polygons(sp_joined_roi.geometry)).opts(
    alpha=1, colorbar=True, clabel="Building footprint density", cmap=cc.fire
)
base_layer_hv * coastal_mask_hv * building_footprints_hv